In [16]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
import optuna

In [17]:
y_train = pd.read_csv("Y_train.csv")
y_train= y_train["mort_icu"]
y_valid = pd.read_csv("Y_valid.csv")
y_valid = y_valid["mort_icu"]

X_train = pd.read_csv("X_train_new.csv")
X_train = X_train.dropna(axis = 1)
X_valid = pd.read_csv("X_valid_new.csv")
X_valid = X_valid.dropna(axis = 1)

X_train = X_train.append(X_valid)
y_train = y_train.append(y_valid)

/tmp/.u3568933/ipykernel_24678/3419343273.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_valid)
/tmp/.u3568933/ipykernel_24678/3419343273.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_valid)


In [18]:
def objective(trial,data=X_train,target=y_train):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'n_neighbors': trial.suggest_categorical('n_neighbors', [1, 3, 5, 7, 9, 11, 13, 15]),
        'weights': trial.suggest_categorical("weights", ["uniform", "distance"]),
        'algorithm': trial.suggest_categorical("algorithm", ["auto", "ball_tree", "kd_tree", "brute"])        
    }
    model = KNeighborsClassifier(**param)  
    
    model.fit(train_x,train_y)
    
    y_valid_scores = model.predict_proba(test_x)[:, 1]
    fpr, tpr, threshold = roc_curve(test_y, y_valid_scores)
    auc_valid = auc(fpr, tpr)

    return auc_valid

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=64)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-11-26 12:56:13,333] A new study created in memory with name: no-name-5d3a7a37-dcc6-47b0-aae7-fcb856226728
[I 2022-11-26 12:56:14,095] Trial 0 finished with value: 0.5850537558850396 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'algorithm': 'auto'}. Best is trial 0 with value: 0.5850537558850396.
[I 2022-11-26 12:56:14,843] Trial 1 finished with value: 0.5850537558850396 and parameters: {'n_neighbors': 1, 'weights': 'uniform', 'algorithm': 'brute'}. Best is trial 0 with value: 0.5850537558850396.
[I 2022-11-26 12:56:34,361] Trial 2 finished with value: 0.7485427939006395 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 2 with value: 0.7485427939006395.
[I 2022-11-26 12:56:35,129] Trial 3 finished with value: 0.6949862975194997 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'brute'}. Best is trial 2 with value: 0.7485427939006395.
[I 2022-11-26 12:56:35,938] Trial 4 finished with value: 0.7512657

Number of finished trials: 64
Best trial: {'n_neighbors': 15, 'weights': 'distance', 'algorithm': 'ball_tree'}
